In [5]:
# Install requirements
! pip install -r requirements.txt

  Using cached pmtiles-3.4.1-py3-none-any.whl

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [22]:
import pystac
from pmtiles.reader import Reader
from datetime import datetime


In [23]:
# Create a catalog and save
catalog = pystac.Catalog(id="pmtiles-catalog", description="PMTiles datasets")
tiles = [
    "static/N50_BygningerOgAnlegg_omrade.pmtiles",
    "static/N50_Samferdsel_senterlinje.pmtiles",
    "static/N50_raster_2024.pmtiles",
]
for tile in tiles:
    # with open(tile) as f:
    #     # Open PMTiles and get metadata
    #     r = Reader(f)
    #     m = r.metadata
    #     print(m)
    bounds = [1,2,3,4]
    minzoom = 0
    maxzoom = 2

    # # Create STAC Item
    item = pystac.Item(
        id="example-pmtiles",
        geometry={
            "type": "Polygon",
            "coordinates": [[
                [bounds[0], bounds[1]],
                [bounds[2], bounds[1]],
                [bounds[2], bounds[3]],
                [bounds[0], bounds[3]],
                [bounds[0], bounds[1]]
            ]]
        },
        bbox=bounds,
        datetime=datetime.now(),
        properties={"minzoom": minzoom, "maxzoom": maxzoom}
    )

    item.add_asset("pmtiles", pystac.Asset(
        href=f"http://localhost:8000/{tile.replace("static/","")}",
        media_type="application/vnd.pmtiles"
    ))

    catalog.add_item(item)
catalog.normalize_and_save("stac-pmtiles", catalog_type=pystac.CatalogType.SELF_CONTAINED)


Run a static STAC catalog

In [ ]:
!docker run \
  --env-file .env \
  --env app_port=8000 \
  --env app_host=0.0.0.0 \
	--env reload=false \
  --env log_level=info \
  --env catalog_href=/var/www/html/static/catalog.json \
  --volume /tmp:/tmp \
  --volume ./stac-pmtiles/:/var/www/html/static \
  --publish 8080:8000 \
  ghcr.io/fntb/stac-fastapi-static:1.0.1

1091.66s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO:     172.17.0.1:42758 - "GET /collections HTTP/1.1" 200 OK
INFO:     172.17.0.1:42774 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     172.17.0.1:42758 - "GET /items HTTP/1.1" 404 Not Found
INFO:     172.17.0.1:42774 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     172.17.0.1:40820 - "GET / HTTP/1.1" 200 OK
INFO:     172.17.0.1:40830 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     Shutting down
^C
INFO:     Finished server process [1]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.12/asyncio/runners.py", line 195, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "u